In [1]:
import pandas as pd

csv_path = "flight_status_raw_20230430_20251213.csv"

df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("행 수:", len(df))
df.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_860\2402491823.py:5: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ CSV 로드 완료
행 수: 1500000


,AIRLINE_ENGLISH,AIRLINE_KOREAN,AIRPORT,AIR_FLN,ARRIVED_ENG,ARRIVED_KOR,BAGGAGE_CLAIM,BOARDING_ENG,BOARDING_KOR,CITY,...,FLIGHT_DATE,GATE,IO,LINE,LINE_CODE,RMK_ENG,RMK_KOR,STD,UFID,flight_date
0,JEJU AIR,제주항공,CJJ,7C802,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,20230430,NaN,I,국내,D,ARRIVED,도착,1040,20230430CJJI7C802,2023-04-30
1,JEJU AIR,제주항공,CJJ,7C850,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,20230430,NaN,I,국내,D,DELAY,지연,2320,20230430CJJI7C850,2023-04-30
2,JEJU AIR,제주항공,CJJ,7C852,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,20230430,NaN,I,국내,D,ARRIVED,도착,920,20230430CJJI7C852,2023-04-30
3,JEJU AIR,제주항공,CJJ,7C854,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,20230430,NaN,I,국내,D,ARRIVED,도착,1840,20230430CJJI7C854,2023-04-30
4,JEJU AIR,제주항공,CJJ,7C856,CHEONGJU,청주,1.0,JEJU,제주,CJU,...,20230430,NaN,I,국내,D,ARRIVED,도착,1810,20230430CJJI7C856,2023-04-30


In [2]:
df["AIRLINE_KOREAN"].unique()

array(['제주항공', '대한항공', '진에어', '아시아나항공', '에어로케이항공', '티웨이항공', '하이에어',
       '플라이강원', '춘추항공', '에어부산', '캐세이퍼시픽항공', '중국길상항공(준야오항공)', '중국동방항공',
       '에어서울', '로얄에어필리핀', '스쿠트타이거항공', '홍콩익스프레스', '이스타항공', '중국국제항공',
       '중화항공', '중국남방항공', '상해항공', '일본항공', '전일본공수', '타이거에어 타이완', '몽골항공',
       '필리핀항공', '비엣젯항공', '베트남항공', '에어아시아엑스', '에바항공(장영항공)', '퍼시픽항공',
       '비엣트래블항공', '사천항공', '이즈니스항공', '룽에어', '훈누항공', '스타플라이어항공', '에어로 몽골리아',
       '프랑스항공', '델타항공', '길상항공(준야오)', '싱가포르항공', '필리핀에어아시아', nan, '스쿠트 항공',
       '벰부항공', '라오항공', '호주항공(콴타스항공)', '말레이시아항공', '알이탈리아항공', '서부항공',
       '사우디항공', '네덜란드항공', '하와이안항공', '북경수도항공', '스카이앙코르항공', '페덱스화물항공',
       '하문항공', '그레이터베이항공', '장시항공', '중국심천항공', '럭키항공', '비티엘항공', '중국연합항공',
       '루일리항공', '칭다오항공', '오케이항공', '원통화물항공', '에어캐나다', '젯스타 에어웨이즈', '말린도항공',
       '천진항공', '동하이항공', '인도항공', '캄보디아항공', '아에로멕시코', 'ITA항공', '에바항공',
       '에어인천', '피치항공', '카놋샤크항공', '세부퍼시픽항공', '에어아스타나항공', '파라타항공',
       '가루다인도네시아'], dtype=object)

In [3]:
df["AIRLINE_KOREAN"].value_counts()


대한항공           274941
아시아나항공         270658
제주항공           180582
진에어            176052
티웨이항공          160994
                ...  
이즈니스항공              2
에어인천                1
말레이시아항공             1
호주항공(콴타스항공)         1
필리핀에어아시아            1
Name: AIRLINE_KOREAN, Length: 85, dtype: int64

In [4]:
df["RMK_KOR"].value_counts()

출발          730096
도착          723601
결항           11434
지연             736
회항             437
수속중            297
탑승중            289
탑승장 입장         154
탑승최종            44
탑승구 변경          21
마감예정            13
수하물 투입 중         8
수속중단             7
미정               4
사전결항             1
Name: RMK_KOR, dtype: int64

In [5]:
#타겟 변수(is_delay) 생성
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)


In [6]:
df["is_delay"].value_counts()

0    1487392
1      12608
Name: is_delay, dtype: int64

In [7]:
import sys
!{sys.executable} -m pip install xlsxwriter


In [8]:
# =====================================================
# ✈️ 항공사별 비행 횟수
# =====================================================
airline_counts = (
    df["AIRLINE_KOREAN"]
    .value_counts(dropna=False)
    .reset_index()
)
airline_counts.columns = ["AIRLINE_KOREAN", "total_flights"]

print("\n✈️ 항공사별 비행 횟수")
print(airline_counts)



✈️ 항공사별 비행 횟수
   AIRLINE_KOREAN  total_flights
0            대한항공         274941
1          아시아나항공         270658
2            제주항공         180582
3             진에어         176052
4           티웨이항공         160994
..            ...            ...
81          비티엘항공              2
82        말레이시아항공              1
83           에어인천              1
84    호주항공(콴타스항공)              1
85       필리핀에어아시아              1

[86 rows x 2 columns]


In [9]:
import pandas as pd

# =====================================================
# 2️⃣ BOARDING_KOR 전처리 (국내 출발지만 + 표준화)
# =====================================================

allowed_boarding_raw = [
    "제주",
    "서울/김포",
    "부산/김해",
    "청주",
    "대구",
    "광주",
    "인천",
    "여수",
    "울산",
    "포항/포항경주",
    "진주/사천",
    "군산",
    "원주",
    "무안",
    "양양"
]

boarding_replace_map = {
    "서울/김포": "김포",
    "부산/김해": "김해",
    "포항/포항경주": "포항경주",
    "진주/사천": "사천"
}

before_rows = len(df)
df = df[df["BOARDING_KOR"].isin(allowed_boarding_raw)].copy()
after_rows = len(df)

print(f"✂️ 출발지 필터링: {before_rows} → {after_rows}")

df["BOARDING_KOR"] = df["BOARDING_KOR"].replace(boarding_replace_map)

print("✅ BOARDING_KOR 전처리 완료")
print(df["BOARDING_KOR"].value_counts())

# =====================================================
# 3️⃣ 타겟 변수 생성 (지연 여부)
# =====================================================
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)

print("\n📌 지연 분포")
print(df["is_delay"].value_counts())

# =====================================================
# 4️⃣ 날짜 파생
# =====================================================
df["flight_date"] = pd.to_datetime(df["FLIGHT_DATE"], format="%Y%m%d")

# =====================================================
# 5️⃣ 출발지 / 도착지 단순 카운트
# =====================================================
boarding_counts = (
    df["BOARDING_KOR"]
    .value_counts(dropna=False)
    .reset_index()
)
boarding_counts.columns = ["BOARDING_KOR", "total_flights"]

arrived_counts = (
    df["ARRIVED_KOR"]
    .value_counts(dropna=False)
    .reset_index()
)
arrived_counts.columns = ["ARRIVED_KOR", "total_flights"]

# =====================================================
# 🔹 5-1️⃣ 항공사별 비행 횟수 (신규)
# =====================================================
airline_counts = (
    df["AIRLINE_KOREAN"]
    .value_counts(dropna=False)
    .reset_index()
)
airline_counts.columns = ["AIRLINE_KOREAN", "total_flights"]

# =====================================================
# ✂️ 도착지 비행 횟수 24회 이상만 필터링
# =====================================================
arrived_counts_24plus = arrived_counts[arrived_counts["total_flights"] >= 24].copy()

print("\n📍 도착지 비행 횟수 24회 이상")
print(arrived_counts_24plus)

# =====================================================
# 6️⃣ 출발지 기준 지연 요약
# =====================================================
boarding_summary = (
    df
    .groupby("BOARDING_KOR")["is_delay"]
    .agg(
        total_flights="count",
        delay_count="sum",
        delay_rate="mean"
    )
    .reset_index()
    .sort_values("delay_count", ascending=False)
)

# =====================================================
# 7️⃣ 도착지 기준 지연 요약
# =====================================================
arrived_summary = (
    df
    .groupby("ARRIVED_KOR")["is_delay"]
    .agg(
        total_flights="count",
        delay_count="sum",
        delay_rate="mean"
    )
    .reset_index()
    .sort_values("delay_count", ascending=False)
)

# =====================================================
# 8️⃣ 엑셀 저장 (전처리된 결과만)
# =====================================================
save_path = "flight_airport_summary.xlsx"

with pd.ExcelWriter(save_path, engine="openpyxl") as writer:
    boarding_counts.to_excel(writer, sheet_name="boarding_counts", index=False)
    arrived_counts.to_excel(writer, sheet_name="arrived_counts", index=False)
    
        # ✅ 신규 추가
    arrived_counts_24plus.to_excel(
        writer,
        sheet_name="arrived_counts_24plus",
        index=False
    )
    
    airline_counts.to_excel(writer, sheet_name="airline_counts", index=False)  # 🔹 추가
    boarding_summary.to_excel(writer, sheet_name="boarding_summary", index=False)
    arrived_summary.to_excel(writer, sheet_name="arrived_summary", index=False)

print(f"\n✅ 엑셀 저장 완료: {save_path}")


✂️ 출발지 필터링: 1500000 → 1313683
✅ BOARDING_KOR 전처리 완료
제주      478130
김포      375396
김해      243947
청주       62358
대구       51161
광주       36107
인천       19596
여수       12982
울산       10977
포항경주      5701
사천        5560
군산        4454
원주        3825
무안        2915
양양         574
Name: BOARDING_KOR, dtype: int64

📌 지연 분포
0    1301822
1      11861
Name: is_delay, dtype: int64

📍 도착지 비행 횟수 24회 이상
    ARRIVED_KOR  total_flights
0            제주         453852
1         서울/김포         323936
2         부산/김해         155092
3            청주          48408
4            대구          39036
..          ...            ...
96        난퉁/난통             28
97           리장             28
98           위린             27
99    방콕/방콕/돈므앙             26
100        나가사키             26

[101 rows x 2 columns]

✅ 엑셀 저장 완료: flight_airport_summary.xlsx


In [10]:
# =====================================================
# ✅ 도착지 24회 이상 리스트
# =====================================================
valid_arrived = arrived_counts_24plus["ARRIVED_KOR"].tolist()

print("✅ 허용 도착지 개수:", len(valid_arrived))
print(valid_arrived)


✅ 허용 도착지 개수: 101
['제주', '서울/김포', '부산/김해', '청주', '대구', '광주', '오사카/간사이', '도쿄/하네다', '인천', '타이페이/타오위안', '상하이/푸동', '여수', '후쿠오카', '울산', '도쿄/나리타', '상하이/홍차오', '다낭', '나트랑', '포항/포항경주', '진주/사천', '군산', '방콕/수완나품', '베이징/서우두', '홍콩', '원주', '삿포로', '세부', '싱가폴/싱가포르', '카오슝/가오슝', '대용/장가계', '베이징(서우두)/서우두', '하노이', '타이페이/쑹산', '청도/칭다오', '괌', '호치민/호찌민', '클락필드/클라크필드', '오키나와', '울란바토르/칭기스칸', '베이징/다싱', '난징(남경)/난징', '항조우/항저우', '연길/옌지', '나고야/나고야/주부센트레아', '마닐라', '나고야/주부', '타그빌라란/보홀(탁빌라란)', '심천/선전', '무안', '심양/선양', '푸꼭(푸국)/푸꾸옥', '닝보', '비엔티안', '베이징(다싱)/다싱', '마카오', '코타키나발루', '마즈야마/마쓰야마', '치앙마이', '양양', '달랏', '방콕/돈므앙', '구마모도/구마모토', '오사카/오사카/간사이', '시안', '덴파사르', '산야/싼야', '팡라오/보홀팡라오', '광저우', '키타큐슈', '쉬지아쭈앙(석가장)/스자좡', '정주/정저우', '도쿄/도쿄/나리타', '오르도스', '쿠알라룸푸르', '푸꼭(푸국)/두옹당(푸?)', '이바라키', '푸저우', '나고야/주부센트에어국제공항', '방콕/방콕/수완나품', '대련/다롄', '우시(무석)/우시', '칼리보', '타이쭝/타이중/칭촨강', '오비히로', '톈진', '히로시마', '사이판', '퉁치/황산', '장사', '장춘/창춘', '씨앰립/시엠립앙코르', '타이유안/타이위안', '제남/지난', '아사이까와/아사히카와', '도야마', '하얼빈', '난퉁/난통', '리장', '위린', '방콕/방콕/돈므앙', '나가사키']


In [11]:
# =====================================================
# ✅ 최종 분석용 데이터 필터링
# =====================================================
before_rows = len(df)

df_final = df[
    (df["BOARDING_KOR"].isin(df["BOARDING_KOR"].unique())) &  # 이미 국내만 남아 있음
    (df["ARRIVED_KOR"].isin(valid_arrived))
].copy()

after_rows = len(df_final)

print(f"📊 최종 데이터 필터링: {before_rows} → {after_rows}")


📊 최종 데이터 필터링: 1313683 → 1313426


In [12]:
print("\n✈️ 최종 출발지 분포")
print(df_final["BOARDING_KOR"].value_counts())

print("\n📍 최종 도착지 분포")
print(df_final["ARRIVED_KOR"].value_counts())

print("\n📌 최종 지연 분포")
print(df_final["is_delay"].value_counts())



✈️ 최종 출발지 분포
제주      478106
김포      375344
김해      243865
청주       62307
대구       51151
광주       36107
인천       19596
여수       12982
울산       10977
포항경주      5701
사천        5560
군산        4454
원주        3825
무안        2877
양양         574
Name: BOARDING_KOR, dtype: int64

📍 최종 도착지 분포
제주           453852
서울/김포        323936
부산/김해        155092
청주            48408
대구            39036
              ...  
난퉁/난통            28
리장               28
위린               27
나가사키             26
방콕/방콕/돈므앙        26
Name: ARRIVED_KOR, Length: 101, dtype: int64

📌 최종 지연 분포
0    1301569
1      11857
Name: is_delay, dtype: int64


In [13]:
df_final.to_csv(
    "flight_status_final_filtered.csv",
    index=False,
    encoding="utf-8-sig"
)
print("✅ 최종 분석용 CSV 저장 완료")


✅ 최종 분석용 CSV 저장 완료


# 최종 분석용 CSV 파일 

CSV 전체
 ↓
BOARDING_KOR ∈ allowed_boarding_raw   ← 여기서 국내만 남김 (핵심)
 ↓
슬래시 표준화 (김포, 김해, 사천…)
 ↓
도착지 24회 이상 필터
 ↓
최종 분석용 df


In [14]:
import pandas as pd

df_weather = pd.read_csv(
    "airport_final.csv",
    encoding="euc-kr"
)


print("✅ 날씨 CSV 로드 완료")
print("행 수:", len(df_weather))
print("컬럼 수:", df_weather.shape[1])

df_weather.head()


✅ 날씨 CSV 로드 완료
행 수: 344003
컬럼 수: 28


C:\Users\Admin\AppData\Local\Temp\ipykernel_860\55525250.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv(


,지점,지점명,일시,일기현상,1층 운량(1/8),1층 운형,1층 운고(FT),기온(°C),이슬점온도(°C),해면기압(hPa),...,중하층운량(10분위),현상번호(국내식),지면온도(°C),풍속_ms,풍향_deg,시정_m,전운량_okta,1층 운고(FT)_결측여부,운형(운형약어),최저운고(100m )
0,156,광주,2023-04-30 01:00:00,없음,0.0,없음,NaN,10.1,7.1,1012.6,...,9.0,0,11.0,1.0,NaN,2000.0,7,1.0,NaN,NaN
1,156,광주,2023-04-30 02:00:00,없음,0.0,없음,NaN,9.8,6.8,1012.3,...,7.0,0,9.4,1.4,NaN,2000.0,6,1.0,NaN,NaN
2,156,광주,2023-04-30 03:00:00,없음,0.0,없음,NaN,9.2,6.8,1012.2,...,2.0,0,8.3,0.9,NaN,2000.0,2,1.0,NaN,NaN
3,156,광주,2023-04-30 04:00:00,없음,0.0,없음,NaN,8.6,6.8,1012.2,...,0.0,0,7.6,0.1,NaN,2000.0,0,1.0,NaN,NaN
4,156,광주,2023-04-30 05:00:00,없음,0.0,없음,NaN,8.5,7.4,1012.6,...,0.0,0,6.9,0.1,NaN,2000.0,0,1.0,NaN,NaN


In [15]:
df_weather.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344003 entries, 0 to 344002
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   지점              344003 non-null  int64  
 1   지점명             344003 non-null  object 
 2   일시              344003 non-null  object 
 3   일기현상            321059 non-null  object 
 4   1층 운량(1/8)      321059 non-null  float64
 5   1층 운형           321059 non-null  object 
 6   1층 운고(FT)       101698 non-null  float64
 7   기온(°C)          343966 non-null  float64
 8   이슬점온도(°C)       343953 non-null  float64
 9   해면기압(hPa)       343956 non-null  float64
 10  현지기압(hPa)       343958 non-null  float64
 11  강수량(mm)         344003 non-null  float64
 12  습도(%)           183508 non-null  float64
 13  증기압(hPa)        183506 non-null  float64
 14  일조(hr)          344003 non-null  float64
 15  일사(MJ/m2)       344003 non-null  float64
 16  적설(cm)          344003 non-null  float64
 17  3시간신적설(cm)

In [16]:
missing_rate = (
    df_weather.isna().mean()
    .sort_values(ascending=False)
    .reset_index()
)
missing_rate.columns = ["column", "missing_rate"]

missing_rate


,column,missing_rate
0,최저운고(100m ),0.963213
1,운형(운형약어),0.962556
2,1층 운고(FT),0.704369
3,풍향_deg,0.533583
4,중하층운량(10분위),0.468723
5,지면온도(°C),0.466970
6,증기압(hPa),0.466557
7,습도(%),0.466551
8,시정_m,0.091586
9,1층 운형,0.066697


In [17]:
numeric_cols = df_weather.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("📊 수치형 컬럼 개수:", len(numeric_cols))
numeric_cols


📊 수치형 컬럼 개수: 23


['지점',
 '1층 운량(1/8)',
 '1층 운고(FT)',
 '기온(°C)',
 '이슬점온도(°C)',
 '해면기압(hPa)',
 '현지기압(hPa)',
 '강수량(mm)',
 '습도(%)',
 '증기압(hPa)',
 '일조(hr)',
 '일사(MJ/m2)',
 '적설(cm)',
 '3시간신적설(cm)',
 '중하층운량(10분위)',
 '현상번호(국내식)',
 '지면온도(°C)',
 '풍속_ms',
 '풍향_deg',
 '시정_m',
 '전운량_okta',
 '1층 운고(FT)_결측여부',
 '최저운고(100m )']

In [18]:
object_cols = df_weather.select_dtypes(include="object").columns.tolist()

object_cols

['지점명', '일시', '일기현상', '1층 운형', '운형(운형약어)']

In [19]:
# 랜덤 샘플 100개 추출
df_weather_sample = df_weather.sample(n=100, random_state=42)

# 엑셀 저장
sample_path = "airport_final_sample_100.xlsx"
df_weather_sample.to_excel(sample_path, index=False)

print(f"✅ 샘플 100개 엑셀 저장 완료: {sample_path}")


✅ 샘플 100개 엑셀 저장 완료: airport_final_sample_100.xlsx


In [20]:
station_list = (
    df_weather["지점명"]
    .dropna()
    .sort_values()
    .unique()
)

print("✅ 지점명 개수:", len(station_list))
station_list


✅ 지점명 개수: 15


array(['광주', '군산', '김포공항', '김해시', '대구', '무안공항', '양양공항', '여수공항', '울산공항',
       '원주', '인천공항', '제주공항', '진주', '청주', '포항'], dtype=object)

In [21]:
# 출발지 목록 추출
boarding_list = (
    df["BOARDING_KOR"]
    .dropna()
    .sort_values()
    .unique()
)

print("✅ BOARDING_KOR 개수:", len(boarding_list))
boarding_list


✅ BOARDING_KOR 개수: 15


array(['광주', '군산', '김포', '김해', '대구', '무안', '사천', '양양', '여수', '울산', '원주',
       '인천', '제주', '청주', '포항경주'], dtype=object)

In [22]:
# weather 지점명 → BOARDING_KOR 기준으로 변환
weather_to_boarding_map = {
    "광주": "광주",
    "대구" : "대구",
    "군산": "군산",
    "김포공항": "김포",
    "김해시": "김해",
    "무안공항": "무안",
    "양양공항": "양양",
    "여수공항": "여수",
    "울산공항": "울산",
    "원주": "원주",
    "인천공항": "인천",
    "제주공항": "제주",
    "청주": "청주",
    "포항": "포항경주",   # ⚠️ 표기 선택
    "진주": "사천"        # ⚠️ 네가 합의한 해석
}


In [23]:
# ✅ 지점명 → BOARDING_KOR 생성 (핵심)
df_weather["BOARDING_KOR"] = df_weather["지점명"].map(weather_to_boarding_map)


In [24]:
df["STD"].isna().sum()

0

In [25]:
# =====================================================
# STD + FLIGHT_DATE → flight_datetime 생성
# =====================================================

# 1️⃣ STD 컬럼 상태 확인
print("STD 컬럼 정보")
df_final["STD"].info()
print("STD 결측 개수:", df["STD"].isna().sum())

# 2️⃣ STD 정규화 (HHMM → 4자리 문자열)
df_final["STD_clean"] = (
    df_final["STD"]
    .fillna(0)
    .astype(int)
    .astype(str)
    .str.zfill(4)
)

# 3️⃣ flight_datetime 생성 (STD 기준)
df_final["flight_datetime"] = pd.to_datetime(
    df_final["FLIGHT_DATE"].astype(str) + df_final["STD_clean"],
    format="%Y%m%d%H%M",
    errors="coerce"
)

# 4️⃣ 생성 결과 검증
print("\nNaT 개수:", df_final["flight_datetime"].isna().sum())

print("\n샘플 확인")
df_final[
    ["FLIGHT_DATE", "STD", "STD_clean", "flight_datetime"]
].sample(10, random_state=42)

STD 컬럼 정보
<class 'pandas.core.series.Series'>
Int64Index: 1313426 entries, 0 to 1499999
Series name: STD
Non-Null Count    Dtype
--------------    -----
1313426 non-null  int64
dtypes: int64(1)
memory usage: 20.0 MB
STD 결측 개수: 0

NaT 개수: 0

샘플 확인


,FLIGHT_DATE,STD,STD_clean,flight_datetime
53629,20230604,940,0940,2023-06-04 09:40:00
78160,20230621,1205,1205,2023-06-21 12:05:00
1278525,20250801,2045,2045,2025-08-01 20:45:00
672284,20240716,1130,1130,2024-07-16 11:30:00
943962,20241225,1150,1150,2024-12-25 11:50:00
950271,20241229,1940,1940,2024-12-29 19:40:00
903587,20241130,1335,1335,2024-11-30 13:35:00
985862,20250121,1555,1555,2025-01-21 15:55:00
904330,20241201,1340,1340,2024-12-01 13:40:00
425530,20240210,1110,1110,2024-02-10 11:10:00


In [26]:
df_weather["일시"].value_counts().head(10)

2023-04-30 01:00:00    15
2025-01-26 16:00:00    15
2025-01-27 12:00:00    15
2025-01-27 11:00:00    15
2025-01-27 10:00:00    15
2025-01-27 09:00:00    15
2025-01-27 08:00:00    15
2025-01-27 07:00:00    15
2025-01-27 06:00:00    15
2025-01-27 05:00:00    15
Name: 일시, dtype: int64

In [27]:
# =====================================================
# weather 일시 → weather_datetime 생성 (필수)
# =====================================================
df_weather["weather_datetime"] = pd.to_datetime(
    df_weather["일시"],
    errors="coerce"
)

In [29]:
# =====================================================
# 1️⃣ 정렬 (필수)
# =====================================================
df_final = df_final.sort_values("flight_datetime")
df_weather = df_weather.sort_values("weather_datetime")

# =====================================================
# 2️⃣ merge_asof
# =====================================================
df_merged = pd.merge_asof(
    df_final,
    df_weather,
    left_on="flight_datetime",
    right_on="weather_datetime",
    left_by="BOARDING_KOR",
    right_by="BOARDING_KOR",
    direction="backward"
)


In [30]:
print("전체 flight 행 수:", len(df_merged))
print(
    "날씨 붙은 행 수:",
    df_merged["weather_datetime"].notna().sum()
)
print(
    "날씨 미부착 행 수:",
    df_merged["weather_datetime"].isna().sum()
)
print(
    "날씨 부착 비율:",
    df_merged["weather_datetime"].notna().mean()
)


전체 flight 행 수: 1313426
날씨 붙은 행 수: 1313426
날씨 미부착 행 수: 0
날씨 부착 비율: 1.0


In [31]:
(
    df_merged
    .groupby("BOARDING_KOR")["weather_datetime"]
    .apply(lambda x: x.notna().mean())
    .sort_values()
)


BOARDING_KOR
광주      1.0
군산      1.0
김포      1.0
김해      1.0
대구      1.0
무안      1.0
사천      1.0
양양      1.0
여수      1.0
울산      1.0
원주      1.0
인천      1.0
제주      1.0
청주      1.0
포항경주    1.0
Name: weather_datetime, dtype: float64

In [32]:
save_path = "flight_weather_merged.csv"

df_merged.to_csv(
    save_path,
    index=False,
    encoding="utf-8-sig"
)

print(f"✅ CSV 저장 완료: {save_path}")


✅ CSV 저장 완료: flight_weather_merged.csv
